In [ ]:
import os
import sys
import numpy as np
import xarray as xr
import cmocean
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
matplotlib.rc_file('../rc_file')
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - matplotlib.pyplot

In [ ]:
sys.path.append("..")
from MOC import calculate_MOC, AMOC_max, approx_lats
from grid import create_tdepth, find_array_idx
from tqdm import notebook
from regions import Atlantic_mask, boolean_mask
from paths import file_ex_ocn_ctrl, path_results, CESM_filename,\
                  file_ex_ocn_lpd, path_data, path_prace
from plotting import discrete_cmap, shifted_color_map
from timeseries import IterateOutputCESM, lowpass
from xr_DataArrays import xr_DZ, xr_DXU
from xr_regression import xr_lintrend, xr_linear_trend

# AMOC plot

In [ ]:
AMOC_ctrl = xr.open_dataarray(f'{path_results}/MOC/AMOC_max_ctrl.nc', decode_times=False)
AMOC_rcp  = xr.open_dataarray(f'{path_results}/MOC/AMOC_max_rcp.nc' , decode_times=False)
AMOC_lpd  = xr.open_dataarray(f'{path_results}/MOC/AMOC_max_lpd.nc' , decode_times=False)
AMOC_lr1  = xr.open_dataarray(f'{path_results}/MOC/AMOC_max_lr1.nc' , decode_times=False)

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(6.4,3), sharey=True, constrained_layout=True)
ax[0].set_ylabel('AMOC  at  26$\!^\circ\!$N, 1000m')
for i, AMOC_max in enumerate([AMOC_ctrl, AMOC_rcp]):
    c = ['C0', 'C1', 'C3'][i]
    ax[0].plot(AMOC_max.time/365-[0,1800,1800][i], AMOC_max, c=c, alpha=.3, lw=.5)
    ax[0].plot(AMOC_max.time[60:-60]/365-[0,1800,1800][i], lowpass(AMOC_max,120)[60:-60],
               c=c, label=['HIGH CTRL','HIGH RCP'][i])
    if i==1:
        ax[0].plot(AMOC_max.time/365-1800, xr_lintrend(AMOC_max)+2, c=c, ls='--')
ax[0].text(240, 6, f'{xr_linear_trend(AMOC_rcp).values*100*365:3.2f}\nSv/100yr', color='C1')

# RAPID_time = 310+np.arange(0,len(RAPID.time))/12/60
# ax[0].plot(RAPID_time, RAPID['moc_mar_hc10'].rolling(time=60, center=True).mean(), c='C6', alpha=.3)
# ax[0].plot(RAPID_time, RAPID['moc_mar_hc10'].rolling(time=60*12*10, center=True).mean(), c='C6', label='RAPID')
    
for i, AMOC_max in enumerate([AMOC_lpd, AMOC_lr1]):
    ax[1].plot(AMOC_max.time/365-[0,1500,800,1500][i], AMOC_max, c=f'C{i}', alpha=.3, lw=.5)
    ax[1].plot(AMOC_max.time[60:-60]/365-[0,1500,800,1500][i], lowpass(AMOC_max,120)[60:-60],
               c=f'C{i}', label=['LOW CTRL','LOW RCP'][i])
    if i in [1,2]:
        ax[1].plot(AMOC_max.time/365-[0,1500,800,1500][i], xr_lintrend(AMOC_max)+2, c=f'C{i}', ls='--')
        ax[1].text([0,540,1100][i], 6, f'{xr_linear_trend(AMOC_max).values*100*365:3.2f}\nSv/100yr', color=f'C{i}')
        
for i in range(2):
    ax[i].set_ylim((0,29.5))
#     ax[i].set_title(['HIGH CESM1.04', 'LOW CESM1.12'][i])
    ax[i].set_xlabel('time [model years]')
    ax[i].legend(frameon=False)
ax[0].text(.95,.05, 'monthly data &\n10 year lowpass', ha='right', transform=ax[0].transAxes, color='grey')

ax[0].set_xlim((95,305))
ax[1].set_xlim((395,605))
plt.savefig(f'{path_results}/paper3/AMOC_low_high_ctrl_rcp')

## $F_{ov}$ plot

In [ ]:
for i, run in enumerate(['ctrl', 'rcp', 'lpd', 'lr1']):
    fn_out = f'{path_prace}/Mov/Mov_monthly_{run}.nc'
    if run=='ctrl':  Mov_ctrl = xr.open_dataarray(fn_out, decode_times=False)
    if run=='rcp' :  Mov_rcp  = xr.open_dataarray(fn_out, decode_times=False)
    if run=='lpd' :  Mov_lpd  = xr.open_dataarray(fn_out, decode_times=False)
    if run=='lr1' :  Mov_lr1  = xr.open_dataarray(fn_out, decode_times=False)

In [ ]:
f, ax = plt.subplots(1,2,figsize=(6.4,3), sharey=True, constrained_layout=True)
ax[0].set_ylabel(r'$F_{\!ov}$  at  34$\!^\circ\!$S  [Sv]')
for i in range(2):  ax[i].axhline(c='grey', lw=.5)
# ax[0].set_xlim(-5,305)
for i, Mov in enumerate([Mov_ctrl, Mov_rcp]):
    ax[0].plot(Mov.time/365-[0,1800][i], Mov, lw=.5, alpha=.3)
    ax[0].plot(Mov.time[30:-30]/365-[0,1800][i], lowpass(Mov,12*5)[30:-30],
               c=f'C{i}', label=['HIGH CTRL','HIGH RCP'][i])
    if i==1:
        ax[0].plot(Mov.time/365-1800, xr_lintrend(Mov)+.05, ls='--', c='C1')
        ax[0].text(200, -.2,f'{xr_linear_trend(Mov).values*365*100:.2f} Sv/100yr', color='C1', fontsize=8)


for i, Mov in enumerate([Mov_lpd, Mov_lr1]):#, Mov_lr2, Mov_ld]):
    ax[1].plot(Mov.time/365-[0,1500][i], Mov, c=f'C{i}', lw=.5, alpha=.3)
    ax[1].plot(Mov.time[30:-30]/365-[0,1500][i], lowpass(Mov,12*5)[30:-30],
               c=f'C{i}', label=['LOW CTRL', 'LOW RCP'][i])
    if i in [1,2]:
        ax[1].plot(Mov.time/365-[0,1500][i], xr_lintrend(Mov)+.05, ls='--', c=f'C{i}')
        ax[1].text([500,620][i-1], .02,f'{xr_linear_trend(Mov).values*365*100:.2f}\nSv/100yr', color=f'C{i}', fontsize=8)
for i in range(2):
#     ax[i].set_title(['HIGH CESM1.04', 'LOW CESM1.12'][i])
    ax[i].legend(frameon=False, loc=2)
    ax[i].set_xlabel('time [model years]')
ax[1].errorbar(x=200, y=-.165, fmt='none', yerr=.115, capsize=3, c='k')
ax[1].text(220,-.165, 'range of observations\n[Weijer et al. (2019)]', va='center', fontsize=8)
plt.tight_layout()
plt.savefig(f'{path_results}/Mov/Fov_34S_HIGH_LOW')

In [ ]:
for i, run in enumerate(['ctrl', 'rcp', 'lpd', 'lr1']):
    fn_out = f'{path_prace}/Mov/FW_SALT_fluxes_{run}.nc'
    if run=='ctrl':  Mov_ctrl = xr.open_dataset(fn_out, decode_times=False)
    if run=='rcp' :  Mov_rcp  = xr.open_dataset(fn_out, decode_times=False)
    if run=='lpd' :  Mov_lpd  = xr.open_dataset(fn_out, decode_times=False)
    if run=='lr1' :  Mov_lr1  = xr.open_dataset(fn_out, decode_times=False)

In [ ]:
f, ax = plt.subplots(1,2, figsize=(6.4,3), sharey=True)
ov, az, deg = r'$_{\!ov}$', r'$_{\!az}$', r'$\!^\circ\!$'
ax[0].set_ylabel(f'F{ov} / F{az}  at  34{deg}S  [Sv]')
for i in range(2):  ax[i].axhline(0, c='k', lw=.5)
ax[0].plot(Mov_ctrl.Fov.time/365/24 , Mov_ctrl.Fov.isel(nlat_u=810).rolling(time=11, center=True).mean(), c='C0', label=f'F{ov} CTRL')
ax[0].plot(Mov_rcp .Fov.time/365+200, Mov_rcp.Fov.isel(nlat_u=810) .rolling(time=11, center=True).mean(), c='C1', label=f'F{ov} RCP')
ax[0].plot(Mov_ctrl.Faz.time/365/24 , Mov_ctrl.Faz.isel(nlat_u=810).rolling(time=11, center=True).mean(), c='C0', label=f'F{az} CTRL', lw=.7)
ax[0].plot(Mov_rcp .Faz.time/365+200, Mov_rcp.Faz.isel(nlat_u=810) .rolling(time=11, center=True).mean(), c='C1', label=f'F{az} RCP', lw=.7)
ax[0].legend(ncol=2, fontsize=8, frameon=False)

ax[1].plot(Mov_lpd.Fov.time/365+154, Mov_lpd.Fov.isel(nlat_u=85).rolling(time=11, center=True).mean(), c='C0')
ax[1].plot(Mov_lr1.Fov.time/365+500, Mov_lr1.Fov.isel(nlat_u=85).rolling(time=11, center=True).mean(), c='C1')
ax[1].plot(Mov_lpd.Faz.time/365+154, Mov_lpd.Faz.isel(nlat_u=85).rolling(time=11, center=True).mean(), c='C0', lw=.7)
ax[1].plot(Mov_lr1.Faz.time/365+500, Mov_lr1.Faz.isel(nlat_u=85).rolling(time=11, center=True).mean(), c='C1', lw=.7)

for i in range(2):  ax[i].set_xlabel('time [model years]')
ax[1].errorbar(x=200, y=-.165, fmt='none', yerr=.115, capsize=3, c='k')
ax[1].text(220,-.165, 'Fov observation range\n[Weijer et al. (2019)]', va='center', fontsize=8)

In [ ]:
(Fov_lpd.isel(time=slice(-100,-1))-Fov_lpd.isel(time=slice(-100,-1)).mean('time')).plot()

In [ ]:
f, ax = plt.subplots(1,2,figsize=(6.4,3), sharey=True, constrained_layout=True)
ax[0].set_ylabel(r'$F_{\!ov}$  at  34$\!^\circ\!$S  [Sv]')
for i in range(2):  ax[i].axhline(c='grey', lw=.5)
# ax[0].set_xlim(-5,305)
for i, Mov in enumerate([Mov_ctrl, Mov_rcp]):
    ax[0].plot(Mov.time/365-[0,1800][i], Mov, lw=.5, alpha=.3)
    ax[0].plot(Mov.time[30:-30]/365-[0,1800][i], lowpass(Mov,12*5)[30:-30],
               c=f'C{i}', label=['HIGH CTRL','HIGH RCP'][i])
    if i==1:
        ax[0].plot(Mov.time/365-1800, xr_lintrend(Mov)+.05, ls='--', c='C1')
        ax[0].text(200, -.2,f'{xr_linear_trend(Mov).values*365*100:.2f} Sv/100yr', color='C1', fontsize=8)


for i, Mov in enumerate([Mov_lpd, Mov_lr1]):#, Mov_lr2, Mov_ld]):
    ax[1].plot(Mov.time/365-[0,1500][i], Mov, c=f'C{i}', lw=.5, alpha=.3)
    ax[1].plot(Mov.time[30:-30]/365-[0,1500][i], lowpass(Mov,12*5)[30:-30],
               c=f'C{i}', label=['LOW CTRL', 'LOW RCP'][i])
    if i in [1,2]:
        ax[1].plot(Mov.time/365-[0,1500][i], xr_lintrend(Mov)+.05, ls='--', c=f'C{i}')
        ax[1].text([500,620][i-1], .02,f'{xr_linear_trend(Mov).values*365*100:.2f}\nSv/100yr', color=f'C{i}', fontsize=8)
for i in range(2):
#     ax[i].set_title(['HIGH CESM1.04', 'LOW CESM1.12'][i])
    ax[i].legend(frameon=False, loc=2)
    ax[i].set_xlabel('time [model years]')
ax[1].errorbar(x=200, y=-.165, fmt='none', yerr=.115, capsize=3, c='k')
ax[1].text(220,-.165, 'range of observations\n[Weijer et al. (2019)]', va='center', fontsize=8)
plt.tight_layout()
plt.savefig(f'{path_results}/Mov/Fov_34S_HIGH_LOW')